<a href="https://colab.research.google.com/github/sirpps/youtube-splitter/blob/main/split_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Youtube audio Splitter
`En:` An code to extract audio from long youtube video or livestream easly, requeriment Pixeldrain api key, tap  [here](https://pixeldrain.com/user/api_keys)

`Id:` Sebuah kode untuk mengekstrak audio dari video panjang youtube secara mudah, bahan yang diperlukan adalah Pixeldrain api key, klik [disini](https://pixeldrain.com/user/api_keys)

In [ ]:
#@title Install requirements
from IPython.display import clear_output as cls
import os
!pip install yt-dlp
!pip install requests
cls()
print("[installer] yt-dlp version:", os.popen("yt-dlp --version").read().strip())
print("[installer] yt-dlp has been installed")

In [ ]:
#@title Setup
#====================Making Path============
#@markdown ---
#@markdown `En:` Enter any youtube url (not yt short, cuz this function to trim long youtube vid)

#@markdown `Id:` Masukan link youtube (bukan yt short, karena kegunaan kode ini untuk memotong video panjang youtube)
youtube_url = "" #@param{type:"string"}
audio_format = "wav" #@param["mp3","wav","opus","aac"]
#@markdown ---
#@markdown `En:` Input with format `hh:mm:ss` if the video under 1 hour just put 00 in the hour selection

#@markdown `Id:` Masukan dengan format `jj:mm:dd` jika video kurang dari 1 jam isi pada bagian jj 00

#@markdown `En:` Note: h is hour, m is minute, s is second

#@markdown `Id:` Catatan: j itu jam, m itu menit, d itu detik
start_time = "00:00:00" #@param{type:"string"}
end_time = "00:00:00" #@param{type:"string"}
#@markdown ---
#@markdown Enter [pixeldrain](https://pixeldrain.com/user/api_keys) api key
pixeldrain_apikey = "" #@param{type:"string"}
print("[info] setup done")

In [ ]:
#@title Downloading and extract audio
print("[youtube] Downloading yt audio")
!yt-dlp -q --progress -f bestaudio --prefer-ffmpeg -x --audio-format {audio_format} {youtube_url}
print("[youtube] Downloading done")
print("[youtube] Renaming file")
!mv *.{audio_format} aud_raw.{audio_format}
print("[youtube] Done All")

In [ ]:
#@title Triming the audio
print("[Trimer] Starting")
!ffmpeg -i *.{audio_format} -ss {start_time} -to {end_time} -c copy aud_out.{audio_format}
cls()
print("[Trimer] Done")

In [28]:
#@title Renaming audio file
import subprocess
import json

vidtitel = subprocess.run(
    ['yt-dlp', '--dump-json', youtube_url],
    capture_output=True,
    text=True
)
try:
    titellama = json.loads(vidtitel.stdout)['title']
    titel = titellama.lower().replace(" ", "-")
    titelbaru = f'{titel}.{audio_format}'
    !mv aud_out.{audio_format} {titelbaru}
except json.JSONDecodeError:
        print("An Error Occurred")

In [ ]:
#@title Uploading to [Pixeldrain](https://pixeldrain.com/)

upload = subprocess.run(
    ['curl', '-T', titelbaru, '-u', ':'+pixeldrain_apikey, 'https://pixeldrain.com/api/file/'],
    capture_output=True,
    text=True
)
#====================Debug============
#print("Output from curl:", upload.stdout)
#print("Error from curl:", upload.stderr)
#====================Making Path============
try:
    file_result_id = json.loads(upload.stdout)['id']
    print("[uploader] Upload done")
    print("file link: https://pixeldrain.com/u/" + file_result_id)
#    print("ID file:", file_result_id)
except json.JSONDecodeError:
    print("An error occured")